In [1]:
import time
import requests
import pandas as pd
from pathlib import Path
from bs4 import BeautifulSoup

In [2]:
BASE_URL = "https://context.reverso.net/bst-web-user/user/favourites/shared"
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:87.0) Gecko/20100101 Firefox/87.0'
}

In [6]:
class ReversoFavs2Anki():
    def __init__(self, username: str, 
                    src_lang: str='de', 
                    trg_lang: str='es', 
                    audio: bool=False) -> None:
        self.username = username
        self.start = 0
        self.length = 50
        self.src_lang = src_lang
        self.trg_lang = trg_lang
        self.audio = audio

    def get_favs(self, username: str, start: int, length: int):
        params = {
            'userName': username,
            'start': start,
            'length': length,
            'order': 10
        }

        try:
            req = requests.get(BASE_URL, params=params, headers=HEADERS)
            req.raise_for_status()

            return req.json()
        except requests.exceptions.HTTPError as e:
            print(e) # TODO: change to logging
            
    def create_csv(self, data):
        src_lang_w = f'{self.src_lang}_word'
        trg_lang_w = f'{self.trg_lang}_word'
        src_lang_s = f'{self.src_lang}_sentence'
        trg_lang_s = f'{self.trg_lang}_sentence'
        src_lang_a = f'{self.src_lang}_audio'
        
        cols_name = [src_lang_w, src_lang_s, trg_lang_w,trg_lang_s, src_lang_a, 'tag']
        words_dict = dict.fromkeys(cols_name, '')
        return words_dict

    def proccess_favs(self):
        data = self.get_favs(self.username, self.start, self.length)
        content = data['results']
        total_results = data['numTotalResults'] 

        if total_results > self.length:
            new_start = self.length
            new_length = total_results - self.length
            data = self.get_favs(self.username, new_start, new_length)
            content.extend(data['results']) 
        
        dic = self.create_csv(content)
        return dic
            

In [7]:
from pprint import pprint
if __name__ == '__main__':
    rev = ReversoFavs2Anki('lmirandam07', True)
    pprint(rev.proccess_favs())

{'True_audio': '',
 'True_sentence': '',
 'True_word': '',
 'es_sentence': '',
 'es_word': '',
 'tag': ''}
